In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Transactions from a bakery - EDA

베이커리 판매 데이터의 탐색적 분석<br>
판매 품목에 대한 베이커리 데이터 [Kaggle - Transaction from a bakery](https://www.kaggle.com/sulmansarwar/transactions-from-a-bakery)

<br>

**분석 목표**
- 베이커리 판매에 영향을 주는 요인 찾기
- 베이커리 판매량 예측하기
- 세트 상품으로 출시하면 좋을 조합 제안하기

<br>

---

<br>

[1. Introduction](##1.Introduction) <br>
[2. Exploration](##2.Exploration) <br>
　[2-1. 시간대별 베이커리 판매량의 추이](###Introduction)<br>
　[2-2. 베이커리의 베스트 셀러 Top 10](###Introduction)<br>
　[2-3. 베스트 셀러의 시간대별 판매량 추이](###Introduction)<br>
　[2-4. 주요 품목과 연관된 품목](###Introduction)<br>
[3. Analysis](##3. Analysis) <br>
[4. Conclusion](##4. Conclusion) <br>
[5. References](##5. References) <br>

## 1. Introduction

베이커리 판매 데이터
- 2016년 10월 30일부터 2017년 4월 9일까지의 판매 데이터
- 총 21293개의 관측치
- 결측치 값은 NONE으로 설정
- 4개의 변수

| 변수명 | 변수 설명 | 자료 형태 |
|  :----------------------------: |  :----------------------------: |  :----------------------------: |
| Date | 거래일 | 날짜(YYYY-MM-DD) |
| Time | 거래시간 | 시간(HH:MM:DD) |
| Transaction | 거래번호 | 거래의 일련번호 |
| Item | 판매품목 | 상품명 |

**필요한 라이브러리 불러오기**

In [2]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import mlxtend as ml

**데이터 확인하기**

In [18]:
bakery = pd.read_csv('../input/transactions-from-a-bakery/BreadBasket_DMS.csv')
bakery.head()

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
2,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam


In [7]:
bakery.shape

(21293, 4)

In [8]:
bakery.dtypes

Date           object
Time           object
Transaction     int64
Item           object
dtype: object

In [11]:
bakery['Item'].value_counts()

Coffee            5471
Bread             3325
Tea               1435
Cake              1025
Pastry             856
                  ... 
Gift voucher         1
Bacon                1
Adjustment           1
Olum & polenta       1
The BART             1
Name: Item, Length: 95, dtype: int64

**결측치 확인하고 처리하기**

결측치가 있는 행은 전체 삭제하여 처리해준다: 총 786개의 데이터 삭제

In [19]:
bakery[bakery['Item'] == 'NONE'].count()

Date           786
Time           786
Transaction    786
Item           786
dtype: int64

In [16]:
bakery = bakery[bakery['Item'] != 'NONE']
bakery[bakery['Item'] == 'NONE']

,Date,Time,Transaction,Item


## 2. Exploration

## 3. Analysis

## 4. Conclusion

## 5. Reference